### Rahul Zalkikar | Predicting Soybean Yield
A quantitative approach

##### BASELINE MODEL

In [58]:
# Imports
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
pd.set_option('display.max_columns', None)

Connect to our repo directory

In [59]:
proj_dir = os.getcwd()+'\\OneDrive\\Pictures\\Documents\\DS-GA1001-Repo\\Veggie-Tales\\final_repo\\'

Our merged dataset contains yearly state feature data representing soil condition, weather and moisture, and geospatial data related to the planting and harvesting of soybean, along with the yield (our target variable) and production value metrics.

This data was primarly compiled through the USDA (soybean related data) and NOAA (climate and weather related data) via API and manual download.

Values of -99.0 represent an instance of no data, and NaN values represent an NA data value.

In [60]:
merged_df = pd.read_csv(proj_dir+'dat\\Processed_Outputs_NA_UPDATE2\\SOIL_DATA_MERGED.csv')

In [61]:
merged_df.head()

YEAR  LOCATION  STATE_ANSI  AREA_OPERATED_IN_ACRES  \
0  1996   ALABAMA           1               9700000.0   
1  1996  ARKANSAS           5              14900000.0   
2  1996  DELAWARE          10                590000.0   
3  1996   FLORIDA          12              10700000.0   
4  1996   GEORGIA          13              11400000.0   

   AREA_OPERATED_IN_ACRES_/_OPERATION  OPERATIONS_IN_OPERATIONS  \
0                                 198                     49000   
1                                 301                     49500   
2                                 211                      2800   
3                                 238                     45000   
4                                 233                     49000   

   AREA_HARVESTED_IN_ACRES  AREA_PLANTED_IN_ACRES  PRODUCTION_IN_BU  \
0                 305000.0               320000.0        10370000.0   
1                3600000.0              3650000.0       109800000.0   
2                 217000.0               220000.0         7595000.0   
3                  45000.0                47000.0         1125000.0   
4                 380000.0               400000.0         7980000.0   

   YIELD_IN_BU_/_ACRE  PRODUCTION_IN_$  PRICE_RECEIVED_IN_$_/_BU  \
0                34.0       76738000.0                      7.40   
1                30.5      755424000.0                      7.36   
2                35.0       54684000.0                      7.20   
3                25.0        7875000.0                      7.00   
4                21.0       53306000.0                      6.87   

   CONDITION_IN_PCT_VERY_POOR  CONDITION_IN_PCT_GOOD  CONDITION_IN_PCT_FAIR  \
0                         1.0                   63.0                   22.0   
1                         3.0                   42.5                   28.5   
2                       -99.0                  -99.0                  -99.0   
3                       -99.0                  -99.0                  -99.0   
4                         2.0                   49.0                   30.0   

   CONDITION_IN_PCT_POOR  CONDITION_IN_PCT_EXCELLENT  \
0                    2.0                         9.5   
1                   12.0                        14.0   
2                  -99.0                       -99.0   
3                  -99.0                       -99.0   
4                    7.0                         6.5   

   CONDITION_5_YEAR_AVG_IN_PCT_EXCELLENT  CONDITION_5_YEAR_AVG_IN_PCT_GOOD  \
0                                    NaN                               NaN   
1                                    NaN                               NaN   
2                                  -99.0                             -99.0   
3                                  -99.0                             -99.0   
4                                    NaN                               NaN   

   CONDITION_5_YEAR_AVG_IN_PCT_POOR  CONDITION_5_YEAR_AVG_IN_PCT_VERY_POOR  \
0                               NaN                                    NaN   
1                               NaN                                    NaN   
2                             -99.0                                  -99.0   
3                             -99.0                                  -99.0   
4                               NaN                                    NaN   

   CONDITION_5_YEAR_AVG_IN_PCT_FAIR  CONDITION_PREVIOUS_YEAR_IN_PCT_POOR  \
0                               NaN                                  NaN   
1                               NaN                                  NaN   
2                             -99.0                                -99.0   
3                             -99.0                                -99.0   
4                               NaN                                  NaN   

   CONDITION_PREVIOUS_YEAR_IN_PCT_GOOD  \
0                                  NaN   
1                                  NaN   
2                                -99.0   
3                                -99.0   
4   

In [62]:
print(merged_df.shape)

(722, 1281)


We have a lot of feature columns over a limited number of yearly soybean yield (per state) row values.

First we have to group by state, sort by year, and then lag features measured after Yield in BU per Acre, features that wouldn't be known at the moment of soybean planting in a real-life predictive use case.

We will also leverage lagged Yield in BU per Acre as a feature.

Then we will fill NaN values with 0s as they likely represent no instances of a given feature, and -99.0 values with NaNs before linearly interpolating and extrpolating their values (across both directions).

Finally, we will drop feature columns that represent the same information on a different scale since our goal is to build a  real predictive model that can generalize well.

In [63]:
merged_df = merged_df.fillna(0)
merged_df = merged_df.replace(-99.0,np.NaN)
print(merged_df.isna().sum().sum())

4989


In [74]:
final_df = pd.DataFrame()

In [75]:
cols_to_lag = ['YIELD_IN_BU_/_ACRE','PRODUCTION_IN_BU','PRODUCTION_IN_$',
              'PRICE_RECEIVED_IN_$_/_BU','Crop Moisture Stress Index (CMSI)']

weather_cols = ['AVG_TEMP_Month','MAX_TEMP_Month','MIN_TEMP_Month','Precipitation_Month','PDSI_Month','PMDI_Month',
                'Z_index_Month','PHDI_Month']
counter = 0
for col in weather_cols:
    for i in range(4,13):
        cols_to_lag.append(col+str(i))

In [76]:
for _,g in merged_df.groupby('LOCATION'):
    g.sort_values('YEAR',ascending=True)
    for col in cols_to_lag:
        g[col+'_lagged'] = g[col].shift(1)
        if col != 'YIELD_IN_BU_/_ACRE':
            del g[col]
    final_df = final_df.append(g)

C:\Users\rayzc\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [77]:
# Lagging a good number of features requires us to drop the year 1996 prior to linear interpolation/extrapolation. 
final_df = final_df[final_df.YEAR != 1996] 
# 2019 was almost entirely extrapolated so we remove it.
final_df = final_df[final_df.YEAR != 2019] 

In [78]:
mis_cols = [col for col in final_df.columns if final_df[col].isna().any()]
for col in mis_cols:
    #Interpolate/Extrapolate -99.0 values (which are now NaN values)
    final_df[col] = final_df[col].interpolate(method='linear', axis=0, limit=None, 
                                              inplace=False, limit_direction='both', 
                                              limit_area=None, downcast=None)

In [79]:
final_df = final_df.drop(['STATE_ANSI','LOCATION','AREA_OPERATED_IN_ACRES',
                          'OPERATIONS_IN_OPERATIONS','AREA_HARVESTED_IN_ACRES',
                          'CONDITION_IN_PCT_EXCELLENT','CONDITION_IN_PCT_FAIR',
                          'CONDITION_IN_PCT_GOOD','CONDITION_IN_PCT_POOR',
                          'CONDITION_IN_PCT_VERY_POOR'],axis=1)

To illustrate our model's ability to forecast the most recent year's yield per state, which is our intended use case, we now segment our final dataframe into a training and validation set (includes data from 1997 to 2017) and a testing set (includes 2018 data).

In [80]:
test_df = final_df[final_df.YEAR == 2018] 
train_val_df = final_df[final_df.YEAR != 2018] 

In [81]:
# we finally drop YEAR since our model should be time independent

final_df = final_df.drop(['YEAR'],axis=1)
test_df = test_df.drop(['YEAR'],axis=1)
train_val_df = train_val_df.drop(['YEAR'],axis=1)

In [82]:
# confirm no nan instances
print(final_df.isna().sum().sum(), test_df.isna().sum().sum(), train_val_df.isna().sum().sum())

0 0 0


In [84]:
# confirm shape consistency
print(test_df.shape, train_val_df.shape, final_df.shape)

(31, 1271) (647, 1271) (678, 1271)


In [86]:
train_val_df.head()

,AREA_OPERATED_IN_ACRES_/_OPERATION,AREA_PLANTED_IN_ACRES,YIELD_IN_BU_/_ACRE,CONDITION_5_YEAR_AVG_IN_PCT_EXCELLENT,CONDITION_5_YEAR_AVG_IN_PCT_GOOD,CONDITION_5_YEAR_AVG_IN_PCT_POOR,CONDITION_5_YEAR_AVG_IN_PCT_VERY_POOR,CONDITION_5_YEAR_AVG_IN_PCT_FAIR,CONDITION_PREVIOUS_YEAR_IN_PCT_POOR,CONDITION_PREVIOUS_YEAR_IN_PCT_GOOD,CONDITION_PREVIOUS_YEAR_IN_PCT_VERY_POOR,CONDITION_PREVIOUS_YEAR_IN_PCT_EXCELLENT,CONDITION_PREVIOUS_YEAR_IN_PCT_FAIR,AVG_TEMP_Month1,AVG_TEMP_Month2,AVG_TEMP_Month3,MAX_TEMP_Month1,MAX_TEMP_Month2,MAX_TEMP_Month3,MIN_TEMP_Month1,MIN_TEMP_Month2,MIN_TEMP_Month3,Precipitation_Month1,Precipitation_Month2,Precipitation_Month3,PDSI_Month1,PDSI_Month2,PDSI_Month3,PMDI_Month1,PMDI_Month2,PMDI_Month3,Z_index_Month1,Z_index_Month2,Z_index_Month3,PHDI_Month1,PHDI_Month2,PHDI_Month3,NITROGEN,PHOSPHATE,POTASH,SULFUR,APPLICATIONS_IN_LB_FERTILIZER_NITROGEN,APPLICATIONS_IN_LB_FERTILIZER_PHOSPHATE,APPLICATIONS_IN_LB_FERTILIZER_POTASH,APPLICATIONS_IN_LB_FERTILIZER_SULFUR,APPLICATIONS_IN_LB_/_ACRE_/_APPLICATION_AVG_FERTILIZER_NITROGEN,APPLICATIONS_IN_LB_/_ACRE_/_APPLICATION_AVG_FERTILIZER_PHOSPHATE,APPLICATIONS_IN_LB_/_ACRE_/_APPLICATION_AVG_FERTILIZER_POTASH,APPLICATIONS_IN_LB_/_ACRE_/_APPLICATION_AVG_FERTILIZER_SULFUR,APPLICATIONS_IN_LB_/_ACRE_/_YEAR_AVG_FERTILIZER_NITROGEN,APPLICATIONS_IN_LB_/_ACRE_/_YEAR_AVG_FERTILIZER_PHOSPHATE,APPLICATIONS_IN_LB_/_ACRE_/_YEAR_AVG_FERTILIZER_POTASH,APPLICATIONS_IN_LB_/_ACRE_/_YEAR_AVG_FERTILIZER_SULFUR,APPLICATIONS_IN_NUMBER_AVG_FERTILIZER_NITROGEN,APPLICATIONS_IN_NUMBER_AVG_FERTILIZER_PHOSPHATE,APPLICATIONS_IN_NUMBER_AVG_FERTILIZER_POTASH,APPLICATIONS_IN_NUMBER_AVG_FERTILIZER_SULFUR,TREATED_IN_PCT_OF_AREA_PLANTED_AVG_FERTILIZER_NITROGEN,TREATED_IN_PCT_OF_AREA_PLANTED_AVG_FERTILIZER_PHOSPHATE,TREATED_IN_PCT_OF_AREA_PLANTED_AVG_FERTILIZER_POTASH,TREATED_IN_PCT_OF_AREA_PLANTED_AVG_FERTILIZER_SULFUR,AZOXYSTROBIN,PYRACLOSTROBIN,PROPICONAZOLE,TRIFLOXYSTROBIN,TEBUCONAZOLE,FLUXAPYROXAD,PROTHIOCONAZOLE,FLUAZINAM,TETRACONAZOLE,MYCLOBUTANIL,FLUOXASTROBIN,THIOPHANATE-METHYL,METCONAZOLE,CHLOROTHALONIL,DIFENOCONAZOLE,PICOXYSTROBIN,CYPROCONAZOLE,FLUOPYRAM,MONO-POTASSIUM SALT,BOSCALID,MEFENOXAM,BENZOVINDIFLUPYR,CONIOTHYRIUM MINITAN,BORAX DECAHYDRATE,ISOFETAMID,SEDAXANE,FLUDIOXONIL,FENHEXAMID,BACILLUS PUMILUS,MANCOZEB,ZOXAMIDE,STREPTOMYCES LYDICUS,CYPRODINIL,COPPER CHLORIDE HYD.,BASIC COPPER SULFATE,APPLICATIONS_IN_LB_FUNGICIDE_AZOXYSTROBIN,APPLICATIONS_IN_LB_FUNGICIDE_PYRACLOSTROBIN,APPLICATIONS_IN_LB_FUNGICIDE_PROPICONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_TRIFLOXYSTROBIN,APPLICATIONS_IN_LB_FUNGICIDE_TEBUCONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_FLUXAPYROXAD,APPLICATIONS_IN_LB_FUNGICIDE_PROTHIOCONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_SULFUR,APPLICATIONS_IN_LB_FUNGICIDE_FLUAZINAM,APPLICATIONS_IN_LB_FUNGICIDE_TETRACONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_MYCLOBUTANIL,APPLICATIONS_IN_LB_FUNGICIDE_FLUOXASTROBIN,APPLICATIONS_IN_LB_FUNGICIDE_THIOPHANATE-METHYL,APPLICATIONS_IN_LB_FUNGICIDE_METCONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_CHLOROTHALONIL,APPLICATIONS_IN_LB_FUNGICIDE_DIFENOCONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_PICOXYSTROBIN,APPLICATIONS_IN_LB_FUNGICIDE_CYPROCONAZOLE,APPLICATIONS_IN_LB_FUNGICIDE_FLUOPYRAM,APPLICATIONS_IN_LB_FUNGICIDE_MONO-POTASSIUM SALT,APPLICATIONS_IN_LB_FUNGICIDE_BOSCALID,APPLICATIONS_IN_LB_FUNGICIDE_MEFENOXAM,APPLICATIONS_IN_LB_FUNGICIDE_BENZOVINDIFLUPYR,APPLICATIONS_IN_LB_FUNGICIDE_CONIOTHYRIUM MINITAN,APPLICATIONS_IN_LB_FUNGICIDE_BORAX DECAHYDRATE,APPLICATIONS_IN_LB_FUNGICIDE_ISOFETAMID,APPLICATIONS_IN_LB_FUNGICIDE_SEDAXANE,APPLICATIONS_IN_LB_FUNGICIDE_FLUDIOXONIL,APPLICATIONS_IN_LB_FUNGICIDE_FENHEXAMID,APPLICATIONS_IN_LB_FUNGICIDE_BACILLUS PUMILUS,APPLICATIONS_IN_LB_FUNGICIDE_MANCOZEB,APPLICATIONS_IN_LB_FUNGICIDE_ZOXAMIDE,APPLICATIONS_IN_LB_FUNGICIDE_STREPTOMYCES LYDICUS,APPLICATIONS_IN_LB_FUNGICIDE_CYPRODINIL,APPLICATIONS_IN_LB_FUNGICIDE_COPPER CHLORIDE HYD.,APPLICATIONS_IN_LB_FUNGICIDE_BASIC COPPER SULFATE,APPLICATIONS_IN_LB_/_ACRE_/_APPLICATION_AVG_FUNGICIDE_AZOXYSTROBIN,APPLICATIONS_IN_LB_/

In [87]:
# Save our data
train_val_df.to_csv(os.getcwd()+'\\OneDrive\\Pictures\\Documents\\DS-GA1001-Repo\\Veggie-Tales\\rahul\\finaldatasets\\train.csv',index=False)
test_df.to_csv(os.getcwd()+'\\OneDrive\\Pictures\\Documents\\DS-GA1001-Repo\\Veggie-Tales\\rahul\\finaldatasets\\test.csv',index=False)
final_df.to_csv(os.getcwd()+'\\OneDrive\\Pictures\\Documents\\DS-GA1001-Repo\\Veggie-Tales\\rahul\\finaldatasets\\final_df.csv',index=False)

In [88]:
target = ['YIELD_IN_BU_/_ACRE']
X = train_val_df.drop(target,axis=1)
y = train_val_df[target].values.reshape(-1)

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
print(train_val_df.shape)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(647, 1271)
(517, 1270) (130, 1270)
(517,) (130,)


In [100]:
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [101]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [102]:
def reg_metrics(y_test,y_pred):
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Standardizing the features around the center and 0 with a standard deviation of 1 is important when we compare measurements that have different units. Variables that are measured at different scales do not contribute equally to the analysis and might end up creating a bais.

In [103]:
std_fit = StandardScaler().fit(X_train)
X_train = std_fit.transform(X_train)
X_test = std_fit.transform(X_test)

C:\Users\rayzc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\rayzc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\rayzc\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


Our first model is going to be a Random Forest Regressor via sklearn.

We leverage randomized search for optimizing our hyperparameters. All parameters that influence the learning are searched simultaneously (except for the number of estimators, which poses a time / quality tradeoff).

For comparisons sake, randomized search and grid search explore exactly the same space of parameters. The result in parameter settings is quite similar, while the run time for randomized search is drastically lower.

For a simple baseline, we will apply 3 fold cross validation on a random search of parameters, across 100 total combinations (using all available cores of a system). This is standard practice for any baseline RF model.

In [104]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] # num trees in a forest
max_features = ['auto', 'sqrt'] # num features to consider at each split
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # max num levels in tree
max_depth.append(None)
min_samples_split = [2, 5, 10] # min samples for a node split
min_samples_leaf = [1, 2, 4] # min samples at each leaf node
bootstrap = [True, False] # how to sample

#grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [105]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 24.3min finished
C:\Users\rayzc\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [106]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 100,
 'bootstrap': True}

In [108]:
clf = RandomForestRegressor(n_estimators=800,min_samples_split=2,min_samples_leaf=1,max_features='auto',max_depth=100,
                            bootstrap=True)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [110]:
for feature in clf.feature_importances_: print(feature)

0.008255446489193255
0.0039337884757385845
0.00483934679831567
0.0026782107498268854
0.002556914960818369
0.006059503254857369
0.003393329114445011
0.0024798794853147215
0.002544293642344615
0.006277300577651126
0.0014982983135523734
0.00298516869447103
0.010970978289474213
0.00221231698248721
0.0018870121100041026
0.023366106732497095
0.0023521232558529224
0.001907639204397899
0.006004217829857489
0.0017876519080645926
0.0020819861314986168
0.007309242894738156
0.0052966525381755525
0.007703249317891749
0.0018039317671420124
0.00309446394280938
0.002396437222404086
0.0012574510435558195
0.0014412250347800228
0.0017882366298128454
0.007861940112500139
0.005524653211206466
0.006154688450648919
0.0010182470281312936
0.0013503370553663472
0.0015725449447708067
3.757763009959969e-05
4.19050017012235e-05
2.829948852872917e-05
7.2270912797777194e-06
0.00017972814123055076
0.0003532108015126521
0.00022826924923470958
6.671013205244576e-05
0.00019920335825948505
0.00025577976326331435
0.000216

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.3487847140787223e-06
1.843187680435246e-07
9.77725746015389e-05
1.2029611821963606e-06
0.0
1.9471400420206542e-07
6.91654254910695e-06
1.2868191039784417e-06
3.4355102909911334e-06
6.3880018237157725e-06
3.5679076890074306e-06
3.1503251975126052e-06
6.396635189224442e-06
4.851501411265031e-06
5.9125030512797536e-08
1.1773935706888454e-05
1.564336077903535e-06
7.565551170382608e-06
7.132908271311668e-06
3.1501995761627236e-06
4.962162814286189e-09
6.527279200172054e-06
0.0
1.3957436007908021e-08
3.596797639573268e-08
8.043526135791885e-09
0.0
4.62392255153309e-06
4.463628930833869e-07
0.0
5.895521463287255e-07
2.8968453648588305e-08
0.0
1.442410759971355e-08
3.6060859680581186e-07
0.0
0.0
0.0
0.0
0.0
0.0
8.195526499555451e-07
3.4430277017796493e-07
0.00016070108488853188
2.5455069744815703e-07
1.351593280118726e-08
1.53642742162439e-07
1.6348997474145762e-07
5.44205923414896

In [111]:
rfr_pred = clf.predict(X_test)

In [112]:
mean_absolute_percentage_error(y_test, rfr_pred)

13.670759953178349

In [113]:
reg_metrics(y_test,rfr_pred)

Mean Absolute Error: 4.274350961538461
Mean Squared Error: 30.230696595552885
Root Mean Squared Error: 5.498244865004912


In [114]:
error_df = pd.DataFrame({'Actual': y_test, 'Predicted': rfr_pred})
error_df.head(25)

,Actual,Predicted
0,44.0,35.700625
1,33.0,37.966875
2,36.5,33.280625
3,35.5,33.578750
4,24.0,28.498125
5,41.5,33.230625
6,27.5,36.223750
7,38.0,38.367500
8,30.0,36.899375
9,37.0,40.906250


In [115]:
# Now for year 2018!
X_t = test_df.drop(target,axis=1)
y_t = test_df[target].values.reshape(-1)
test_pred = clf.predict(X_t)
mean_absolute_percentage_error(y_t, test_pred)

30.84325319984297